In this notebook we will implement 2 models that do some calculations on encrypted data

In [4]:
from collections import namedtuple
import secrets 
from typing import List,Tuple 
import numpy as np
from crypto import RandomPrime, xgcd, LCM,InverseMod
PrivateKey = namedtuple("Private_key", ["lam",'mu'])
PublicKey = namedtuple("Public_key",["g","n",'n_sqr'])
DEFAULT_BIT_SIZE = 64
def L(x:int,n:int) -> int:
    return (x-1)//n

In [107]:
# we are going to use this class to work with the methods of Paillier implemented previousely
class Paillier(): 
    def __init__(self,size_bits:int = DEFAULT_BIT_SIZE): 
        self.bit_size = size_bits
        p = RandomPrime(size_bits,60)
        q = RandomPrime(size_bits,60)
        while p==q : 
            q = RandomPrime(size_bits,60)
        N = p*q
        phi = (p-1) * (q-1)
        gcd,_,_ = xgcd(N,phi)  
        while gcd!=1:
            p = RandomPrime(size_bits,60)
            q = RandomPrime(size_bits,60)
            while p==q : 
                q = RandomPrime(size_bits,60)
            N = p*q
            phi = (p-1) * (q-1)
            gcd,_,_ = xgcd(N,phi) 

        l = LCM((q-1),(p-1))
        g = secrets.randbelow(N*N) 
        mu = InverseMod(L(pow(g,l,N*N),N),N)

        self.public_key = PublicKey(g,N,N*N)
        self.private_key = PrivateKey(l,mu)
    def encrypt(self,plain_text): 
        g,N, N_sqr = self.public_key
        r = secrets.randbelow(N)
        gcd,_,_ = xgcd(r,N)
        while gcd !=1:
            r = secrets.randbelow(N)
            gcd,_,_ = xgcd(r,N)


        cipher_text = (pow(g,plain_text,N_sqr) * pow(r,N,N_sqr))% (N_sqr)

        return cipher_text
    def decrypt(self,cipher_text): 
        l , mu = self.private_key
        _,N, N_sqr = self.public_key
        recovred_message = (L(pow(cipher_text,l,N_sqr),N)* mu) % N
        return recovred_message
    def add(self,cipher_text_1 : int, cipher_text_2 : int)-> int: 
        return (cipher_text_1 * cipher_text_2 ) % self.public_key.n_sqr
    
    def multiply(self,cipher_text_1 : int, plain_text_2 : int)-> int: 
        #we have 2 cases : 
        if plain_text_2==0:
            #if the plain_text_2 is 0 so everything multiplying by 0 is 0 so we encrypt the 0
            return self.encrypt(0)
        if plain_text_2==1:
            #if the plain_text_2 is 1 so everything multiplying by 1 it stays like ti is so it's equivalent of adding 0
            encryted_0 = self.encrypt(0)
            return self.add(cipher_text_1,encryted_0)
        return pow(cipher_text_1,plain_text_2,self.public_key.n_sqr)
    def __str__(self):
        output = f"""The Paillier parameters:\nPublic key :\n\tg:{self.public_key.g}\n\tn:{self.public_key.n}
        \nPrivate key:\n\tmu:{self.private_key.mu}\n\tlamda: {self.private_key.lam}"""
        return output
paillier = Paillier(DEFAULT_BIT_SIZE)

In [108]:
def encrypted_celsius_to_fahrenheit(paillier: Paillier, cipher_text: int) -> int:
    """
    Returns an encrypted integer representing 1/10ths of a degree Fahrenheit
    °F = °C * 1.8 + 32
    """
    # here we will multiply the equation by 10 so we get 10*°F = 10*°C * 1.8 + 10*32 = °C * 18 + 320
    # so we propose the solution that : we encrypt 320 and we add it to the value of °C * 18 
    encrypted_320= paillier.encrypt(320) 
    return paillier.add(paillier.multiply(cipher_text,18),encrypted_320) 


def encrypted_price_calculator(
    paillier: Paillier,
    # a list of (encrypted price, plaintext quantity) tuples
    encrypted_cart: List[Tuple[int, int]],
) -> int:
    """
    Returns the encrypted sum of multiplying each encrypted price by an unencrypted quantity
    """
    total = paillier.encrypt(0)
    for product in encrypted_cart: 
        price = paillier.multiply(product[0],product[1])
        total = paillier.add(total,price)
    return total


In [114]:
cart_list = [(140,10),(1000,2),(500,4),(330,1)]
expected_total=0
encrypted_cart_list = []
for i in cart_list: 
    price = i[0]*i[1]
    expected_total +=price 
    encrypted_price = paillier.encrypt(i[0])
    encrypted_cart_list.append((encrypted_price,i[1]))
    
encrypted_cart_list

[(14181568848939404498902867607233971855533579070837627401309482109827221888589,
  10),
 (28191448844081729903578176801560590341619326645049600633159221030501579511912,
  2),
 (28899949171375117408915798686590319054202204061274186003217091814283387801254,
  4),
 (24184735169296290149559084548162713333309788876257075873288030589262145722763,
  1)]

# Testing Section

In [116]:
TEST_BIT_LENGTH = 32


def test_encrypted_celsius_to_fahrenheit(paillier: Paillier):
    temperature_in_celsius = 23
    temperature_in_fahrenheit = 73.4

    encrypted_input = paillier.encrypt(temperature_in_celsius)
    encrypted_output = encrypted_celsius_to_fahrenheit(paillier,encrypted_input)
    decrypted_output = paillier.decrypt(encrypted_output)
    # Note that the Paillier cryptosystem only deals with integers and cannot handle
    # encrypted division because it is only a partial homomorphic encryption scheme
    scaled_output = decrypted_output / 10

    assert scaled_output == temperature_in_fahrenheit


def test_encrypted_price_calculator(paillier: Paillier):
    cart = [
        # (price, quantity)
        (2000, 1),
        (120, 5),
        (1999, 3),
    ]
    expected_price = 8597

    # Note that the items are somewhat anonymized by the removal of the highly
    # identifying price information, but the plaintext quantity could still provide
    # information which could deanonymize the item data
    encrypted_cart = [(paillier.encrypt(price), quantity) for price, quantity in cart]
    encrypted_price = encrypted_price_calculator(paillier, encrypted_cart)
    decrypted_price = paillier.decrypt(encrypted_price)

    assert decrypted_price == expected_price


In [117]:
test_encrypted_celsius_to_fahrenheit(paillier)

In [118]:
test_encrypted_price_calculator(paillier)

We note that there is no error so the test is passed sucssessfuly  